In [7]:
import numpy as np
from core.Models import Model
from core.nn import Linear, Conv2d, MaxPool2d , ConvBatchNorm2D,Flatten
from core.Function import Relu,Softmax
from core.optim import sgd, adam
from core.loss import get_loss_fn

class ResNetWithResiduals(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(input_channels=1, output_channels=8, kernel_size=3, stride=1, padding=1, initialize_type='xavier',bias=False)
        self.bn1 = ConvBatchNorm2D(8)
        self.relu1 = Relu()
        self.conv2 = Conv2d(input_channels=8, output_channels=16, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn2 = ConvBatchNorm2D(16)
        self.relu2 = Relu()
        self.res1 = Conv2d(input_channels=8, output_channels=16, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max1 = MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = Conv2d(input_channels=16, output_channels=32, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn3 = ConvBatchNorm2D(32)
        self.relu3 = Relu()
        self.conv4 = Conv2d(input_channels=32, output_channels=64, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn4 = ConvBatchNorm2D(64)
        self.relu4 = Relu()
        self.res2 = Conv2d(input_channels=16, output_channels=64, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max2 = MaxPool2d(kernel_size=2, stride=2)
        self.flatten = Flatten()
        self.linear1 = Linear(64 * 7 * 7, 100, initialize_type='xavier')
        self.linear2 = Linear(100, 10, initialize_type='xavier')
        self.softmax = Softmax()


    def forward(self, x):
        x1 = self.conv1(x)
        x1 = self.bn1(x1)
        x1 = self.relu1(x1)
        x2 = self.conv2(x1)
        x2 = self.bn2(x2)
        x2 = self.relu2(x2)
        res1 = self.res1(x1)  # Match channel dimensions
        x2 = x2 + res1  # Add residual connection
        x2 = self.max1(x2)
        x3 = self.conv3(x2)
        x3 = self.bn3(x3)
        x3 = self.relu3(x3)
        x4 = self.conv4(x3)
        x4 = self.bn4(x4)
        x4 = self.relu4(x4)
        res2 = self.res2(x2)  # Match channel dimensions
        x4 = x4 + res2  # Add residual connection
        x4 = self.max2(x4)
        x4 = self.flatten(x4)
        x4 = self.linear1(x4)
        x4 = self.linear2(x4)
        x4 = self.softmax(x4)
        return x4

# Instantiate model
model_resnet_residuals = ResNetWithResiduals()


In [8]:
from core.Datasets import mnist
train_dataset, test_dataset = mnist(batch_size=32)


In [ ]:
# train_dataset.reset()
# x, y = train_dataset.__next__()
# print("Input shape:", y.shape)
# out = model_resnet_residuals(x)
# print("Output shape:", out.shape)
# loss = out.sum()
# loss.backward(retain_graph=True)
# # out.view_graph(filename="model_graph", view=True)



Input shape: (32, 10)
Output shape: (32, 10)


In [ ]:
# loss.visualize()

Graph saved as graph.html


In [ ]:
# print("Output shape:", out.shape)  # Should be (batch_size, 10)

In [9]:
optimizer = adam(model_resnet_residuals.parameters(), learning_rate=0.001)
loss_fn = get_loss_fn('cat_cross_entropy')
num_epochs = 1
def train():
    for epoch in range(num_epochs):
        train_dataset.reset()  # Reset dataset iterator and reshuffle if needed
        epoch_loss = 0.0
        num_batches = 0
        for X_batch, y_batch in train_dataset:
            optimizer.zero_grad()
            out = model_resnet_residuals(X_batch)
            loss_tensor = loss_fn(y_batch, out)
            epoch_loss += loss_tensor.data
            num_batches += 1
            loss_tensor.backward()
            optimizer.step()
        avg_loss = epoch_loss / num_batches
        
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")




In [10]:
train()

Epoch 1/1 - Loss: 0.3309


In [ ]:
def test():
    model_resnet_residuals.test()
    test_dataset.reset()
    correct = 0
    total = 0
    for X_batch, y_batch in test_dataset:
        out = model_resnet_residuals(X_batch)
        predictions = np.argmax(out.data, axis=1)

        # Convert one-hot y_batch back to class indices
        true_labels = np.argmax(y_batch.data, axis=1)

        correct += np.sum(predictions == true_labels)
        total += y_batch.data.shape[0]
    print(f"Accuracy: {correct / total}")


test()

Accuracy: 0.9415714285714286


In [ ]:
from core.Function import *

lr = LeakyReLU(0.01)
x = Tensor.randn(2,3)  # assuming you've implemented randn in your Tensor class
x.requires_grad = True
y = lr(x)
loss = y.sum()          # assuming sum() is implemented
loss.backward()
print("Input gradient shape:", loss.grad)
# Don't access y.grad — it's cleared like PyTorch


# Augmentation Module


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 (only training data for demonstration)
(x_train, y_train), _ = cifar10.load_data()

# Select a sample image (CIFAR-10 images are 32x32 RGB)
sample_img = x_train[0]

# Import Compose and all Preprocessing transforms from your package.
from Preprocessing.transforms import (
    Compose, GaussianBlur, GaussianNoise, Normalize, 
    RandomAffine, RandomCrop, RandomErasing, RandomHorizontalFlip, 
    RandomRotation, RandomVerticalFlip, Resize, ToTensor
)

# Create a dictionary mapping transformation names to their instances.
# For demonstration, we force application by setting probabilities to 1.0.
aug_transforms = {
    "RandomHorizontalFlip": RandomHorizontalFlip(p=1.0),
    "RandomVerticalFlip":   RandomVerticalFlip(p=1.0),
    "RandomCrop":           RandomCrop((28, 28)),  # Crop to 28x28 (from 32x32)
    "RandomRotation":       RandomRotation(degrees=30),
    "RandomAffine":         RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    "GaussianBlur":         GaussianBlur(sigma=(0.5, 1.5)),
    "GaussianNoise":        GaussianNoise(std=(0.01, 0.05)),
    "RandomErasing":        RandomErasing(p=1.0, scale=(0.02, 0.15), ratio=(0.3, 3.3), value=0)
}

# Prepare the plot: one row per Preprocessing (each row shows Original and Augmented)
n_transforms = len(aug_transforms)
fig, axes = plt.subplots(nrows=n_transforms, ncols=2, figsize=(8, n_transforms * 3))

for idx, (name, transform) in enumerate(aug_transforms.items()):
    # Create a Compose pipeline with a single transform.
    pipeline = Compose([transform])
    
    # Left column: display the original image.
    axes[idx, 0].imshow(sample_img)
    axes[idx, 0].set_title("Original")
    axes[idx, 0].axis("off")
    
    # Right column: apply the Compose pipeline and display the result.
    aug_img = pipeline(sample_img)
    if hasattr(aug_img, "data"):
        aug_img = aug_img.data
    axes[idx, 1].imshow(np.clip(aug_img, 0, 255).astype(np.uint8))
    axes[idx, 1].set_title(name)
    axes[idx, 1].axis("off")

plt.tight_layout()
plt.show()


# testing pretrained models

In [ ]:
#https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/

from pretrained.resnet18 import resnet18
from core.Models import Model
model = resnet18(pretrained=True)
model.test()

In [ ]:
from PIL import Image
import numpy as np
from core.tensor import Tensor

# Correct class label for verification
true_label = 153  # Maltese dog

# ImageNet mean and std (RGB order)
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])

# Load and preprocess the image
image_path = r"C:\Users\ahmed\Downloads\n02085936_Maltese_dog (1).JPEG"
img = Image.open(image_path).convert('RGB')  # ensure 3 channels
img = img.resize((224, 224))

# Convert to numpy and normalize
img = np.array(img).astype(np.float32) / 255.0  # scale to [0, 1]
img = (img - mean) / std  # normalize using ImageNet stats

# Convert to (B, C, H, W)
img = np.transpose(img, (2, 0, 1))  # from HWC to CHW
img = np.expand_dims(img, axis=0)   # add batch dim
img = Tensor(img)  # wrap in core.tensor.Tensor

# Run inference
output = model(img)
logits = output.data[0]

# Top-5 predictions
top5 = np.argsort(logits)[::-1][:5]
print("Top-5 predicted class indices:", top5)
print("Is correct class (153) in Top-5:", true_label in top5)


In [ ]:
from pretrained.vgg16 import VGG16
vgg = VGG16(pretrained=True)

In [ ]:
from PIL import Image
import numpy as np
from core.tensor import Tensor

# Correct class label for verification
true_label = 153  # Maltese dog

# ImageNet mean and std (RGB order)
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])

# Load and preprocess the image
image_path = r"C:\Users\ahmed\Downloads\n02085936_Maltese_dog (1).JPEG"
img = Image.open(image_path).convert('RGB')  # ensure 3 channels
img = img.resize((224, 224))

# Convert to numpy and normalize
img = np.array(img).astype(np.float32) / 255.0  # scale to [0, 1]
img = (img - mean) / std  # normalize using ImageNet stats

# Convert to (B, C, H, W)
img = np.transpose(img, (2, 0, 1))  # from HWC to CHW
img = np.expand_dims(img, axis=0)   # add batch dim
img = Tensor(img)  # wrap in core.tensor.Tensor

# Run inference
output = vgg(img)
logits = output.data[0]

# Top-5 predictions
top5 = np.argsort(logits)[::-1][:5]
print("Top-5 predicted class indices:", top5)
print("Is correct class (153) in Top-5:", true_label in top5)


# new layers

In [ ]:
from core.Models import Model
from core.nn import PositionalEmbedding , PatchEmbedding , LayerNorm,MultiHeadAttention

class model(Model):
    def __init__(self):
        super().__init__()
        self.patch_embedding = PatchEmbedding()
        self.positional_embedding = PositionalEmbedding(self.patch_embedding.n_patches, 768)
        self.layer_norm = LayerNorm(768)
        # self.attention = MultiHeadAttention(768, 8,masked=True)


    def forward(self, x):
        x = self.patch_embedding(x)
        x = self.positional_embedding(x)
        x = self.layer_norm(x)
        # x = self.attention(x)
        return x

In [ ]:
from core.tensor import Tensor
import numpy as np
x = Tensor(np.random.rand(1, 3, 224, 224))
model = model()
output = model(x)
output.backward()
print("Output shape:", output.shape)
print(f"output gradient shape: {output.grad.shape}")

In [ ]:
output.view_graph(filename="model_graph", view=True)

# Autograd

In [ ]:
from core.new_tensor import Tensor
import numpy as np


In [ ]:
#numpy array of size 2x2
x_np = np.array([[1,1],[1,1]])
x = Tensor(x_np,requires_grad=True)
z = x + 2
f = x * z
print(f)
# print(f._grad_fn.next_functions)
f.backward(retain_graph=True)
# print(f._engine.graph_nodes)
print(x.grad)


In [ ]:
f.visualize()

In [ ]:
x = Tensor(np.ones((2,2)),requires_grad=True)

# x is used twice in the graph
y1 = x + 1
y2 = x * 3
z = y1 + y2  # final output

z.backward(retain_graph=True)
print("x.grad:")
print(x.grad)

In [ ]:
z.visualize()

In [ ]:
for node in z._engine.graph_nodes:
    print(node)

In [ ]:
import torch

x = torch.ones((2, 2), requires_grad=True)

y1 = x + 1        # y1 = x + 1 → dy1/dx = 1
y2 = x * 3        # y2 = x * 3 → dy2/dx = 3

z = y1 + y2       # z = (x + 1) + (x * 3) = x + 1 + 3x = 4x + 1

z.backward(torch.ones_like(z))

print("x.grad:")
print(x.grad)


In [ ]:
a = Tensor([2.0], requires_grad=True)
b = Tensor([3.0], requires_grad=True)
c = a * b
c.backward()

print("a.grad:", a.grad)  # should be b.data = 3.0
print("b.grad:", b.grad)  # should be a.data = 2.0


In [ ]:
a = Tensor([6.0], requires_grad=True)
b = Tensor([2.0], requires_grad=True)
c = a / 2
print(c)
c.backward()

print("a.grad:", a.grad)  # should be 1 / b = 0.5


In [ ]:
a = Tensor([5.0], requires_grad=True)
b = Tensor([3.0], requires_grad=True)
c = a - b
c.backward()

print("a.grad:", a.grad)  # should be 1
print("b.grad:", b.grad)  # should be -1


In [ ]:
from core.new_tensor import Tensor
import numpy as np

a = Tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
c = a.mean()
c.backward(retain_graph=True)
print("a.grad:")  #
print(a.grad)


In [ ]:
a = Tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
c = a.sum()
c.backward()

print("a.grad:")  # should be all ones
print(a.grad)


In [ ]:
a = Tensor([1.0, 2.0, 3.0], requires_grad=True)
c = a.std()
c.backward()

print("a.grad:")  # should compute ∂std/∂x for each element
print(a.grad)


In [ ]:
from core.new_tensor import Tensor
a = Tensor(2.0, requires_grad=True)
b = Tensor(3.0, requires_grad=True)
c = Tensor(4.0, requires_grad=True)

z = a*b + a*c
z.backward()

print("dz/da =", a.grad)  # expected b + c = 3 + 4 = 7
print("dz/db =", b.grad)  # expected a = 2
print("dz/dc =", c.grad)  # expected a = 2